## nii to png

In [3]:
import nibabel as nib
import numpy as np
import SimpleITK as sitk
import matplotlib.pyplot as plt
import os, cv2
from pathlib import Path
# name = 'TrainValid_Image\\1_Train,Valid_Image\\'
name_m = 'TrainValid_Mask\\2_Train,Valid_Mask\\'
name = 'Test1_Image\\3_Test1_Image\\'
Image_path = f'.\\dataset\\input_img\\aocr2024\\{name}'
mask_path = f'.\\dataset\\input_img\\aocr2024\\{name_m}'

Image_f = os.listdir(Image_path)
mask_f = os.listdir(mask_path)
name_f = Path(Image_f[0]).stem

path = f'.\\dataset\\input_img\\aocr2024\\aocr2024dataset\\'

if not os.path.isdir(path):
    os.mkdir(path)

In [4]:
for i in range(len(Image_f)):
    image = sitk.ReadImage(Image_path + Image_f[i])
    # mask = sitk.ReadImage(mask_path + mask_f[i])

    img_arr = sitk.GetArrayFromImage(image)
    # mask_arr = sitk.GetArrayFromImage(mask)

    name_f = Path(Image_f[i]).stem
    save_path = path + name_f

    if not os.path.isdir(save_path):
        os.mkdir(save_path)
    if not os.path.isdir(save_path + '/Image/'):
        os.mkdir(save_path + '/Image/')
    # if not os.path.isdir(save_path + '/mask/'):
    #     os.mkdir(save_path + '/mask/')


    for idx in range(img_arr.shape[0]):
        fname = str(idx).zfill(3) + '.png'

        plt.imsave(os.path.join(save_path + '/Image/',fname), img_arr[idx], cmap = 'gray')
        # plt.imsave(os.path.join(save_path + '/mask/',fname), mask_arr[idx], cmap = 'gray')

## Test Dataset Load

In [7]:
import csv, shutil, os
csv_path = '.\\dataset\\input_img\\aocr2024\\TrainValid_ground_truth.csv'
path = '.\\dataset\\input_img\\aocr2024\\aocr2024dataset\\'
total_path = os.listdir(path)
move_path = '.\\dataset\\input_img\\aocr2024\\test\\'
for img_type in total_path:
    each_path = os.listdir(f"{path}{img_type}\\Image\\")
    for img in each_path:
        p = f"{path}{img_type}\\Image\\{img}"
        save_p = f"{move_path}{img_type[:-4]}_{int(img[:3])}.png"
        shutil.copyfile(p, save_p)


### Split Train & Validation dataset

In [1]:
import csv, shutil
def file_copy(path, save_path, file_type, file_num, K, dic):
    file_split = {}
    read = csv.reader(open('.\\dataset\\input_img\\aocr2024\\TrainValid_split.csv', 'r'))
    for id, label, group in read:
        file_split[id] = group
    for i in range(len(file_type)):
        if file_split[file_type[i]] == 'Train':
            save_p = f'{save_path}train\\{dic[K]}\\'
        elif file_split[file_type[i]] == 'Valid':
            save_p = f'{save_path}val\\{dic[K]}\\'
        else:
            print('error')
            continue
        save_p = f'{save_p}{file_type[i]}_{file_num[i]}.png'
        p = f'{path}{file_type[i]}.nii\\Image\\{file_num[i]:03d}.png'
        shutil.copyfile(p, save_p)
            

In [2]:
csv_path = '.\\dataset\\input_img\\aocr2024\\TrainValid_ground_truth.csv'
path = '.\\dataset\\input_img\\aocr2024\\aocr2024dataset_1\\'
move_path = '.\\dataset\\input_img\\aocr2024\\'
dict = {1: 'appendicitis', 2: 'normal'}
read_csv = csv.reader(open(csv_path, 'r'))
positive_nm, positive_num, negative_nm, negative_num = [], [], [], []
for id, label in read_csv:
    if label == '1' and ('_' in id):
        index = id.index('_')
        positive_nm.append(id[:index])
        positive_num.append(int(id[index+1:]))
    elif label == '0' and ('_' in id):
        index = id.index('_')
        negative_nm.append(id[:index])
        negative_num.append(int(id[index+1:]))
file_copy(path, move_path, positive_nm, positive_num, 1, dict)
file_copy(path, move_path, negative_nm, negative_num, 2, dict)

### Train & Validation dataset of mask

In [8]:
import shutil, os
load_name, save_name = ['train_seg', 'val_seg'], ['train_mask', 'val_mask']
from util.tool import CreateNF
for i in range(len(load_name)):
    load_path = f'.\\dataset\\input_img\\aocr2024\\{load_name[i]}\\'
    move_path = f'.\\dataset\\input_img\\aocr2024\\{save_name[i]}\\'
    CreateNF(move_path)
    total_name = os.listdir(load_path)
    for nm in total_name:
        name, number = nm.split('_')[0], int(nm.split('_')[1].split('.')[0])
        check_path = f'.\\dataset\\input_img\\aocr2024\\aocr2024dataset_1\\{name}.nii\\mask\\{number:03d}.png'
        m_path = f'{move_path}{nm}'
        shutil.copyfile(check_path, m_path)

### change color range ( 32 to 24 or 8 bits)

In [4]:
from PIL import Image
import os
from util.tool import CreateNF
# load_name, save_name = ['train_seg', 'val_seg'], ['train_seg_8bits', 'val_seg_8bits']
load_name, save_name = ['train_mask_24bits', 'val_mask_24bits'], ['train_mask_8bits', 'val_mask_8bits']
for i in range(len(load_name)):
    load_path = f'.\\dataset\\input_img\\aocr2024\\{load_name[i]}\\'
    move_path = f'.\\dataset\\input_img\\aocr2024\\{save_name[i]}\\'
    CreateNF(move_path)
    total_img_name = os.listdir(load_path)
    for nm in total_img_name:
        img = Image.open(f'{load_path}{nm}')
        # 32 bits to 24 bits
        # img = img.convert('RGB')
        # 24 bits to 8 bits
        img = img.convert('P', palette = Image.ADAPTIVE)
        # 32 bits to 8 bits
        # img = img.convert('P', palette = Image.ADAPTIVE, colors = 16)
        img.save(f'{move_path}{nm}')

### change color range (32 bits)

In [3]:
from PIL import Image
import os
import numpy as np
from util.tool import CreateNF
load_name, save_name = ['train_seg_ori', 'val_seg_ori'], ['train_seg_32bits', 'val_seg_32bits']
for i in range(len(load_name)):
    load_path = f'.\\dataset\\input_img\\aocr2024\\{load_name[i]}\\'
    move_path = f'.\\dataset\\input_img\\aocr2024\\{save_name[i]}\\'
    CreateNF(move_path)
    total_img_name = os.listdir(load_path)
    for nm in total_img_name:
        img = Image.open(f'{load_path}{nm}')
        img = img.convert('RGB')
        img.save(f'{move_path}{nm}')


### 0 1 change

In [1]:
import csv
input_path = '.\\dataset\\input_img\\aocr2024\\csv_result_updated\\'
updated_rows = []
with open(f'{input_path}AOCR2024_test_results.csv', mode='r', newline='') as file:
    reader = csv.reader(file)
    for row in reader:
        if '_' in row[0]:
            if row[1] == '1':
                row[1] = 0
            elif row[1] == '0':
                row[1] = 1
        elif '_' not in row[0] and row[1] == '1':
            row[1] = 0
        updated_rows.append(row)
with open(f'{input_path}updated_AOCR2024_test_results.csv', mode='w', newline='') as file:
    writer = csv.writer(file)
    writer.writerows(updated_rows)



In [2]:
prefixes_with_label_1 = set()
with open(f'{input_path}updated_AOCR2024_test_results.csv', mode='r', newline='') as file:
    reader = csv.reader(file)
    for row in reader:
        img_name = row[0]
        label = row[1]
        if '_' in img_name and label == '1':
            prefix = img_name.split('_')[0]
            prefixes_with_label_1.add(prefix)
updated_rows = []
with open(f'{input_path}updated_AOCR2024_test_results.csv', mode='r', newline='') as file:
    reader = csv.reader(file)
    for row in reader:
        img_name = row[0]
        prefix = img_name.split('_')[0]
        if prefix in prefixes_with_label_1 and '_' not in img_name:
            # print(row)
            row[1] = '1'
        updated_rows.append(row)
        
with open(f'{input_path}Final_updated_AOCR2024_test_results.csv', mode='w', newline='') as file:
    writer = csv.writer(file)
    writer.writerows(updated_rows)

### Check ratio

In [3]:
with open(f'{input_path}Final_updated_AOCR2024_test_results.csv', mode='r', newline='') as file:
    reader = csv.reader(file)
    po, ne = 0, 0
    for row in reader:
        if '_' not in row[0] and row[1] == '1':
            po += 1
        elif '_' not in row[0] and row[1] == '0':
            ne += 1
    print(po, ne)

187 13
